<a href="https://colab.research.google.com/github/dfridland/PyTorch/blob/HW6/PyTorch6Lesson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

conv_1 = torch.nn.Conv1d(1, 1, kernel_size=5, bias=False)
conv_1.weight = torch.nn.Parameter(torch.FloatTensor([[[0.2, 0.2, 0.2, 0.2, 0.2]]]))

conv_1.weight, conv_1.bias

(Parameter containing:
 tensor([[[0.2000, 0.2000, 0.2000, 0.2000, 0.2000]]], requires_grad=True),
 None)

In [2]:
tens = torch.FloatTensor([[93, 1655, 16, 288, 4, 163]])
conv_1(tens[None])

tensor([[[411.2000, 425.2000]]], grad_fn=<ConvolutionBackward0>)

In [3]:
conv_1 = torch.nn.Conv1d(1, 2, kernel_size=5, bias=False)
conv_1.weight = torch.nn.Parameter(torch.FloatTensor([[[0.1, 0.1, 0.1, 0.1, 0.1],
                                                       [0.1, 0.1, 0.1, 0.1, 0.1]
                                                       ]]))

conv_1.weight, conv_1.bias

(Parameter containing:
 tensor([[[0.1000, 0.1000, 0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000, 0.1000, 0.1000]]], requires_grad=True),
 None)

In [4]:
tens = torch.FloatTensor([[0.13, -0.11], [0.24, -0.19], [-0.01, -0.05], [0.07, -0.01],[0.01, 0.01]]).permute(1, 0)
conv_1(tens)

tensor([[0.0090]], grad_fn=<SqueezeBackward1>)

In [5]:
tens.shape

torch.Size([2, 5])

In [6]:
max_words = 2000
max_len = 20
num_classes = 1

#Training
epochs= 5
batch_size = 512
print_batch_n =  100


In [7]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip

--2023-04-11 15:46:03--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Resolving drive.google.com (drive.google.com)... 209.85.146.100, 209.85.146.113, 209.85.146.102, ...
Connecting to drive.google.com (drive.google.com)|209.85.146.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gb2dlb978mei02808sp42p02nnlpe6k6/1681227900000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=68aacd09-ac21-47e4-8ca2-fec7243edaf4 [following]
--2023-04-11 15:46:07--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gb2dlb978mei02808sp42p02nnlpe6k6/1681227900000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=68aacd09-ac21-47e4-8ca2-fec7243edaf4
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 142.250.152.132, 2607

In [8]:
!unzip data.zip

Archive:  data.zip
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               
replace val.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: val.csv                 


In [9]:
import pandas as pd

df_train = pd.read_csv('train.csv')
df_val = pd.read_csv('val.csv')

In [10]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [11]:
df_train['class'].value_counts()

1    92063
0    89404
Name: class, dtype: int64

In [12]:
df_val.tail()

,id,text,class
22678,204145,А я знаю из-за кого такая паршивая погода была...,1
22679,204146,ааа.... что с Гмейлом. не ужто он умер сегодня...,0
22680,204147,"в учебники химии написано ""Пруст Жозеф Луи""\nя...",1
22681,204148,"Вот все любят,а я не люблю..как дура..говорю,ч...",0
22682,204149,уххх спасибо дорогой Леле Евгеньевне и Штепуху...,1


In [13]:
df_val['class'].value_counts()

1    11449
0    11234
Name: class, dtype: int64

In [14]:
#preprocessing

!pip install stop-words pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [16]:
sw = set(get_stop_words('ru'))
sw

{'а',
 'алло',
 'без',
 'близко',
 'более',
 'больше',
 'будем',
 'будет',
 'будете',
 'будешь',
 'будто',
 'буду',
 'будут',
 'будь',
 'бы',
 'бывает',
 'бывь',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'важная',
 'важное',
 'важные',
 'важный',
 'вам',
 'вами',
 'вас',
 'ваш',
 'ваша',
 'ваше',
 'ваши',
 'вверх',
 'вдали',
 'вдруг',
 'ведь',
 'везде',
 'весь',
 'вниз',
 'внизу',
 'во',
 'вокруг',
 'вон',
 'восемнадцатый',
 'восемнадцать',
 'восемь',
 'восьмой',
 'вот',
 'впрочем',
 'времени',
 'время',
 'все',
 'всегда',
 'всего',
 'всем',
 'всеми',
 'всему',
 'всех',
 'всею',
 'всю',
 'всюду',
 'вся',
 'всё',
 'второй',
 'вы',
 'г',
 'где',
 'говорил',
 'говорит',
 'год',
 'года',
 'году',
 'да',
 'давно',
 'даже',
 'далеко',
 'дальше',
 'даром',
 'два',
 'двадцатый',
 'двадцать',
 'две',
 'двенадцатый',
 'двенадцать',
 'двух',
 'девятнадцатый',
 'девятнадцать',
 'девятый',
 'девять',
 'действительно',
 'дел',
 'день',
 'десятый',
 'десять',
 'для',
 'до',
 'довольно',
 'до

In [17]:
puncts = set(punctuation)
puncts

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~'}

In [18]:
morpher = MorphAnalyzer()
morpher.parse('позавтракали')[0].normal_form

'позавтракать'

In [19]:
morpher = MorphAnalyzer()

def preprocess_text(txt):
  txt = str(txt)
  txt = ''.join(c for c in txt if c not in puncts)
  txt = txt.lower()
  txt = re.sub('не\s', 'не', txt)
  txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
  return ' '.join(txt)

In [20]:
df_train['text'].iloc[:1].values

array(['@alisachachka не уезжаааааааай. :(❤ я тоже не хочу, чтобы ты уезжала.'],
      dtype=object)

In [21]:
df_train['text'].iloc[:1].apply(preprocess_text).values

array(['alisachachka неуезжаааааааать ❤ нехотеть уезжать'], dtype=object)

In [22]:
from tqdm import tqdm
tqdm.pandas()

df_train['text'] = df_train['text'].progress_apply(preprocess_text)

df_val['text'] = df_val['text'].progress_apply(preprocess_text)

100%|██████████| 22683/22683 [00:31<00:00, 720.08it/s]


In [23]:
train_corpus = ''.join(df_train['text'])
train_corpus = train_corpus.lower()

In [24]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
tokens = word_tokenize(train_corpus)
tokens[:5]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['alisachachka', 'неуезжаааааааать', '❤', 'нехотеть', 'уезжатьrt']

In [25]:
token_filtered = [word for word in tokens if word.isalnum()]

In [26]:
from nltk.probability import FreqDist
dist = FreqDist(token_filtered)
token_filtered_top = [pair[0] for pair in  dist.most_common(max_words-1)]
len(token_filtered_top)

1999

In [27]:
token_filtered_top[:10]

['хотеть',
 'd',
 'новый',
 'мой',
 'вообще',
 'завтра',
 'хороший',
 'день',
 'немочь',
 'любить']

In [28]:
vocabulary = {v: k for k, v in dict(enumerate(token_filtered_top, 1)).items()}
vocabulary

{'хотеть': 1,
 'd': 2,
 'новый': 3,
 'мой': 4,
 'вообще': 5,
 'завтра': 6,
 'хороший': 7,
 'день': 8,
 'немочь': 9,
 'любить': 10,
 'делать': 11,
 'самый': 12,
 'думать': 13,
 'смотреть': 14,
 'идти': 15,
 'дом': 16,
 '2': 17,
 'один': 18,
 'друг': 19,
 'знать': 20,
 '3': 21,
 'такой': 22,
 'спать': 23,
 'утро': 24,
 'незнать': 25,
 'час': 26,
 'человек': 27,
 'школа': 28,
 'сидеть': 29,
 'сделать': 30,
 'небыть': 31,
 'писать': 32,
 'пойти': 33,
 'свой': 34,
 'ждать': 35,
 'любимый': 36,
 'блин': 37,
 'настроение': 38,
 'мама': 39,
 'никто': 40,
 'работа': 41,
 'нехотеть': 42,
 'ночь': 43,
 'написать': 44,
 'последний': 45,
 'говорить': 46,
 'какой': 47,
 'неделя': 48,
 'чтоть': 49,
 'болеть': 50,
 'мочь': 51,
 'фильм': 52,
 'посмотреть': 53,
 'первый': 54,
 'твой': 55,
 'добрый': 56,
 'урок': 57,
 'купить': 58,
 'снег': 59,
 'скоро': 60,
 'голова': 61,
 'пара': 62,
 'домой': 63,
 'равно': 64,
 'прийти': 65,
 'найти': 66,
 'дело': 67,
 'стать': 68,
 'давать': 69,
 'rt': 70,
 '5': 71,


In [29]:
import numpy as np

def text_to_sequence(text, maxlen):
  result=[]
  tokens = word_tokenize(text.lower())
  tokens_filtered = [word for word in tokens if word.isalnum()]
  for word in tokens_filtered:
    if word in vocabulary:
      result.append(vocabulary[word])
  padding = [0] * (maxlen - len(result))
  return result[-maxlen:] + padding

In [30]:
%%time
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train['text']])
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val['text']])


CPU times: user 24.4 s, sys: 133 ms, total: 24.5 s
Wall time: 27.4 s


In [31]:
x_train.shape

(181467, 20)

In [32]:
df_train['text'].iloc[0]

'alisachachka неуезжаааааааать ❤ нехотеть уезжать'

In [33]:
x_train[0]

array([ 42, 628,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0])